# <h1><center>Interactive Spatial Data Visualization</center></h1>

## 5.4 - IpyLeaflet

In [1]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
                        LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
                        MarkerCluster, Heatmap,SearchControl, 
                        FullScreenControl)

from ipywidgets import Text, HTML
from branca.colormap import linear
import geopandas as gpd
import json

In [2]:
# Countries
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Conflict Dataset Points
asia_acled = gpd.read_file(
    "data/acled2019.shp",
    mask = countries[countries["continent"] == "Asia"]
)


In [3]:
asia_acled.head()

,data_id,iso,event_id_c,event_id_n,event_date,year,time_preci,event_type,sub_event_,actor1,...,latitude,longitude,geo_precis,source,source_sca,notes,fatalities,timestamp,iso3,geometry
0,6768128,887,YEM45982,45982,31 December 2019,2019,1,Battles,Armed clash,Military Forces of Yemen (2012-),...,14.3541,47.0765,2,Yemen Data Project; Aden al Ghad; Al Janoob al...,Local partner-National,"On 31 December 2019, clashes reportedly erupte...",0,1580761030,YEM,POINT (47.07650 14.35410)
1,6768129,887,YEM45983,45983,31 December 2019,2019,1,Strategic developments,Change to group/activity,Military Forces of Yemen (2012-) Security Belt...,...,14.3541,47.0765,2,Al Janoob al Youm; Yemen Data Project,Local partner-National,"On 31 December 2019, forces loyal to the South...",0,1580761030,YEM,POINT (47.07650 14.35410)
2,6768130,887,YEM45984,45984,31 December 2019,2019,1,Battles,Armed clash,Military Forces of Yemen (2012-),...,14.2081,47.1570,1,Al Janoob al Youm; Aden al Ghad; Yemen Data Pr...,Local partner-National,"On 31 December 2019, clashes reportedly erupte...",1,1580761030,YEM,POINT (47.15700 14.20810)
3,6715395,356,IND68948,68948,31 December 2019,2019,1,Protests,Peaceful protest,Protesters (India),...,20.2724,85.8338,1,Pioneer (India),National,"On 31 December, students' wing of INC and Mili...",0,1578514389,IND,POINT (85.83380 20.27240)
4,6714375,356,IND68961,68961,31 December 2019,2019,1,Protests,Peaceful protest,Protesters (India),...,25.2905,79.8753,1,Amar Ujala,Subnational,"On Dec 31, protests were held by auto drivers ...",0,1578514386,IND,POINT (79.87530 25.29050)


## Base maps, Markers & Icons


In [4]:
center = [57.71, 11.98]
zoom = 12

m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m

Map(center=[57.71, 11.98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [5]:
stamen = basemap_to_tiles(basemaps.Stamen.Toner)
m.add_layer(stamen)

In [6]:
marker = Marker(location=center, draggable=False)
m.add_layer(marker);

m

Map(center=[57.71, 11.98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [7]:
icon_url = "http://icons.iconarchive.com/icons/pelfusion/long-shadow-media/512/Maps-Pin-Place-icon.png"

icon = Icon(icon_url=icon_url)
mark = Marker(location=[57.719503, 12.008843], icon=icon, rotation_angle=0,  rotation_origin='1280px 128px')
m.add_layer(mark);
m

Map(center=[57.71, 11.98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

## Geodata and Map Styles

In [8]:
center = [4.546647, 22.373178]
zoom = 3


africa = countries[countries["continent"] == "Africa"]
africaMap = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
geodata = GeoData(geo_dataframe = africa, 
                   style={'color': 'black', 'fillColor': '#E0D071', 'opacity':0.03, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': '#b08a3e' , 'fillOpacity': 0.8},
                   name = 'Countries')


africaMap.add_layer(geodata)
africaMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [9]:
html = HTML('''Hover Over Countries''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
africaMap.add_control(control)

def update_html(feature,  **kwargs):
    html.value = '''
        <h3><b>{}</b></h3>
        <h4>GDP: {:.2e} people</h4> 
        <h4>Population: {}</h4>
    '''.format(feature['properties']['name'],
               feature['properties']['gdp_md_est'],
               feature['properties']['pop_est'])

geodata.on_hover(update_html)

In [10]:
africa.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
11,83301151,Africa,Dem. Rep. Congo,COD,66010.0,"POLYGON ((29.34000 -4.49998, 29.51999 -5.41998..."
12,7531386,Africa,Somalia,SOM,4719.0,"POLYGON ((41.58513 -1.68325, 40.99300 -0.85829..."
13,47615739,Africa,Kenya,KEN,152700.0,"POLYGON ((39.20222 -4.67677, 37.76690 -3.67712..."


### Choropleth Map

In [11]:
africa.at[60,'name']="Ivory Coast"
pop =  dict(zip(africa['name'].tolist(), africa['pop_est'].tolist()))
africa[["name", "geometry"]].to_file("data/africa.json", driver="GeoJSON", encoding='utf-8')
geojson_data = json.load(open("data/africa.json",'r'))

In [12]:
for feature in geojson_data['features']:
    properties = feature['properties']
    feature.update(id=properties['name'])
    #print(feature['id'])

In [13]:
choroplethData = Choropleth(
    geo_data=geojson_data,
    choro_data=pop,
    colormap=linear.YlOrRd_04,
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
)

In [14]:
choro_map = Map(center = (4.546647, 22.373178), zoom = 3)
choro_map.add_layer(choroplethData)
choro_map

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

### Clustered Maps

In [15]:
pointsMap = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
PointData = GeoData(geo_dataframe = asia_acled.sample(500), name = 'Countries')


pointsMap.add_layer(PointData)
pointsMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [16]:

clusterMap = Map(center=center, zoom=zoom)
clusterMap.add_layer(MarkerCluster(
    markers=[Marker(location=geolocation.coords[0][::-1]) for geolocation in asia_acled.sample(1000).geometry])
    )
clusterMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Controls

In [17]:
search = SearchControl(position="topleft", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=5,
                       property_name='display_name',
                       marker=marker
                      )
clusterMap.add_control(search)

clusterMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [18]:
control = FullScreenControl()
clusterMap.add_control(control)